<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.4 MB/s eta 0:00:00


In [3]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50): #cria 50 linhas
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+------------------------------------------------------+--------------------------+----------+-------------------------+----------------+---------------------+
|address                                               |date                      |dob       |email                    |name            |phone                |
+------------------------------------------------------+--------------------------+----------+-------------------------+----------------+---------------------+
|500 Gonzalez Plaza\nChristopherfort, WA 12756         |2024-05-02 06:13:38.727743|1913-08-19|jonespamela@example.org  |Daniel Lowery   |386-510-9816         |
|87024 Katherine Flat\nSouth Phillipchester, PW 07085  |2024-05-03 04:15:02.234841|1972-10-10|jasonthompson@example.org|Shannon Graham  |001-807-624-1514x3217|
|556 Scott Locks\nDustinmouth, NY 50670                |2024-05-01 16:35:26.89966 |1975-07-05|mwilliams@example.com    |Tammy Rose      |(765)254-9222x0589   |
|51662 Brent Loop Apt. 842\nMichelleport

# Writing as PARQUET



In [4]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-dce7cf18-8f8a-4de6-a8b3-a15f8d32b2ae-c000.snappy.parquet  _SUCCESS


50

In [8]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd")) #separa em folders com o nome da data e coloca lá os ficheiros particionados

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe
                                                                    # NÃO USAR STATIC NUNCA

(df#.where("date_part = '20240503'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240502'  'date_part=20240503'  'date_part=20240504'


50

In [9]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+-------------------+--------------------+
|             address|                date|       dob|               email|               name|               phone|
+--------------------+--------------------+----------+--------------------+-------------------+--------------------+
|500 Gonzalez Plaz...|2024-05-02 06:13:...|1913-08-19|jonespamela@examp...|      Daniel Lowery|        386-510-9816|
|4852 Aaron Mounta...|2024-05-02 05:09:...|1984-12-10|  jperez@example.net|    Jennifer Haynes|       (694)714-8587|
|624 Sandra Lakes ...|2024-05-02 20:35:...|1983-07-18| etaylor@example.net|   Michael Chambers| +1-559-362-4026x363|
|683 Briggs Tunnel...|2024-05-02 12:41:...|1927-01-21|  ladams@example.net|         Mark Allen|          5778127169|
|PSC 0273, Box 378...|2024-05-02 09:20:...|1947-09-07|  mike26@example.org|      Whitney Black|+1-312-435-9026x6...|
|496 Cole Landing\...|2024-05-02 06:01:...|1987-12-04|  ngarza@e

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [10]:
df.count()

50

In [11]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-985451ca-b900-48a2-ac64-5c6c8e25e43d-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [13]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save(path))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-a4834821-8902-4a82-b89d-c3f520350c7b-c000.json  _SUCCESS


50

In [14]:
# reading json as text
spark.read.text(path).show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                     |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"500 Gonzalez Plaza\nChristopherfort, WA 12756","date":"2024-05-02T06:13:38.727Z","dob":"1913-08-19","email":"jonespamela@example.org","name":"Daniel Lowery","phone":"386-510-9816","date_part":"20240502"}                   |
|{"address":"87024 Katherine Flat\nSouth Phillipches

In [15]:
# reading json as text
spark.read.json(path).show(10, False)

+------------------------------------------------------+------------------------+---------+----------+-------------------------+----------------+---------------------+
|address                                               |date                    |date_part|dob       |email                    |name            |phone                |
+------------------------------------------------------+------------------------+---------+----------+-------------------------+----------------+---------------------+
|500 Gonzalez Plaza\nChristopherfort, WA 12756         |2024-05-02T06:13:38.727Z|20240502 |1913-08-19|jonespamela@example.org  |Daniel Lowery   |386-510-9816         |
|87024 Katherine Flat\nSouth Phillipchester, PW 07085  |2024-05-03T04:15:02.234Z|20240503 |1972-10-10|jasonthompson@example.org|Shannon Graham  |001-807-624-1514x3217|
|556 Scott Locks\nDustinmouth, NY 50670                |2024-05-01T16:35:26.899Z|20240501 |1975-07-05|mwilliams@example.com    |Tammy Rose      |(765)254-9222x0

In [17]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
print(spark.table("tbl_json_part").count())

# read as json
spark.sql("show partitions tbl_json_part").show()

50
+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



In [18]:
spark.sql("show tables from default").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|tbl_json_part|      false|
+---------+-------------+-----------+



# Append Mode

In [19]:
# Writing as PARQUET with APPEND - ATENÇAO AOS DUPLICADOS pq o append tá sempre a adicionar linhas

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-02fc0145-ba69-450c-9e6b-7e754bb4e45e-c000.snappy.parquet  _SUCCESS


50

In [ ]:
/parquet_folder/20240602

#Forma Spark de particionar : Hive-style
/parquet_folder/date_part=20240602